In [1]:
import gym
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import time
import pandas as pd
from gym import wrappers
from gym import spaces
import math
import random

In [2]:
use_custom=True

In [3]:

open('newfile.txt','w').writelines([ line for line in open('messagesoutput1.txt') if 'fq_calculate_drop_prob-start' in line])
csvfile = open('myfile.txt', 'w')

file1 = open('newfile.txt', 'r')
Lines = file1.readlines()
 
count = 0
# Strips the newline character
for line in Lines:
    count += 1
    temp=line.split("-")
    temp1=temp[1]
    
    temp1=temp1.split(",")
    if(len(temp1)==24):
        csvfile.writelines(temp[1])
    
csvfile.close()



df=pd.read_csv("myfile.txt",names=["start","q->flow_index","pprms->qdelay_ref","pprms->tupdate","pprms->max_burst","pprms->max_ecnth","pprms->alpha","pprms->beta","pprms->flags","pst->burst_allowance","pst->drop_prob","pst->current_qdelay","pst->qdelay_old","pst->accu_prob","pst->measurement_start","pst->avg_dq_time","pst->dq_count","pst->sflags","q->stats.tot_pkts","q->stats.tot_bytes","q->stats.length","q->stats.len_bytes","q->stats.drops","end"],header=None)
df=df.drop(columns=['start', 'end'])
df['ecn'] = df['q->flow_index'].map(lambda x: x>2)
df["ecn"] = df["ecn"].astype(int)
df=df.drop(columns=['q->flow_index'])

In [4]:
df = df.loc[:,df.apply(pd.Series.nunique) != 1]

In [5]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
  
# copy the data
df_sklearn = df.copy()
  
# apply normalization techniques
column = 'pst->burst_allowance'
df_sklearn[column] = MinMaxScaler().fit_transform(np.array(df_sklearn[column]).reshape(-1,1))



column = 'pst->current_qdelay'
df_sklearn[column] = MinMaxScaler().fit_transform(np.array(df_sklearn[column]).reshape(-1,1))


column = 'pst->qdelay_old'
df_sklearn[column] = MinMaxScaler().fit_transform(np.array(df_sklearn[column]).reshape(-1,1))


column = 'pst->accu_prob'
df_sklearn[column] = MinMaxScaler().fit_transform(np.array(df_sklearn[column]).reshape(-1,1))


column = 'pst->measurement_start'
df_sklearn[column] = MinMaxScaler().fit_transform(np.array(df_sklearn[column]).reshape(-1,1))

column = 'q->stats.tot_pkts'
df_sklearn[column] = MinMaxScaler().fit_transform(np.array(df_sklearn[column]).reshape(-1,1))
column = 'q->stats.tot_bytes'
df_sklearn[column] = MinMaxScaler().fit_transform(np.array(df_sklearn[column]).reshape(-1,1))
column = 'q->stats.length'
df_sklearn[column] = MinMaxScaler().fit_transform(np.array(df_sklearn[column]).reshape(-1,1))
column = 'q->stats.len_bytes'
df_sklearn[column] = MinMaxScaler().fit_transform(np.array(df_sklearn[column]).reshape(-1,1))
column = 'q->stats.drops'
df_sklearn[column] = MinMaxScaler().fit_transform(np.array(df_sklearn[column]).reshape(-1,1))

  




# column = 'pst->drop_prob'
# df_sklearn[column] = MinMaxScaler().fit_transform(np.array(df_sklearn[column]).reshape(-1,1))


column1 = 'pst->drop_prob'
column1_max=df_sklearn[column1].abs().max()
df_sklearn[column1] = df_sklearn[column1] /df_sklearn[column1].abs().max()


# view normalized data  
display(df_sklearn)

,pst->burst_allowance,pst->drop_prob,pst->current_qdelay,pst->qdelay_old,pst->accu_prob,pst->measurement_start,q->stats.tot_pkts,q->stats.tot_bytes,q->stats.length,q->stats.len_bytes,q->stats.drops,ecn
0,1.000000,8.882767e-06,0.026316,0.026316,0.000000,0.000000,0.005324,0.004520,0.027559,0.027559,0.000000,1
1,1.000000,8.882767e-06,0.026316,0.026316,0.000000,0.000201,0.007800,0.006996,0.043307,0.043307,0.000000,1
2,1.000000,8.882767e-06,0.026316,0.026316,0.000000,0.001003,0.016343,0.015537,0.059055,0.059055,0.000000,1
3,0.888889,7.012898e-06,0.026316,0.026316,0.000000,0.001003,0.017209,0.016404,0.055118,0.055118,0.000000,1
4,1.000000,8.882767e-06,0.026316,0.026316,0.000000,0.001672,0.026495,0.025689,0.141732,0.141732,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...
3260,0.000000,1.150208e-02,0.236842,0.236842,0.008377,0.998328,0.536462,0.531163,0.090551,0.090551,0.176471,1
3261,0.000000,1.329746e-02,0.236842,0.236842,0.008377,0.998328,0.536462,0.531163,0.090551,0.090551,0.176471,1
3262,0.000000,0.000000e+00,0.078947,0.078947,0.052158,0.998328,0.540052,0.534752,0.110236,0.110207,0.176471,1
3263,0.000000,4.672887e-07,0.052632,0.052632,0.052158,0.998328,0.540052,0.534752,0.047244,0.047215,0.176471,1


In [6]:
df_sklearn.rename(columns={'pst->burst_allowance': 'burst_allowance', 
'pst->drop_prob': 'drop_prob',
'pst->current_qdelay': 'current_qdelay',
'pst->qdelay_old': 'qdelay_old',
'pst->accu_prob': 'accu_prob',
'pst->measurement_start': 'measurement_start',
'q->stats.tot_pkts': 'tot_pkts',
'q->stats.tot_bytes': 'tot_bytes',
'q->stats.length': 'length',
'q->stats.len_bytes': 'len_bytes',
'q->stats.drops': 'drops'
}, inplace=True)

In [7]:
input=df_sklearn["drop_prob"][0]
df_sort = df_sklearn.iloc[(df_sklearn['drop_prob']-input).abs().argsort()[:1]]

In [8]:
df_sort["drop_prob"].values[0]

8.882766564563791e-06

In [9]:
print(df_sklearn["drop_prob"].abs().max())
print(df_sklearn["burst_allowance"].abs().max())
print(df_sklearn["current_qdelay"].abs().max())
print(df_sklearn["qdelay_old"].abs().max())
print(df_sklearn["accu_prob"].abs().max())
print(df_sklearn["measurement_start"].abs().max())
print(df_sklearn["tot_pkts"].abs().max())
print(df_sklearn["tot_bytes"].abs().max())
print(df_sklearn["length"].abs().max())
print(df_sklearn["len_bytes"].abs().max())
print(df_sklearn["drops"].abs().max())
print(df_sklearn["ecn"].abs().max())

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0000000000000002
1.0
1.0
1.0
1


In [10]:
print(df_sklearn["drop_prob"].abs().min())
print(df_sklearn["burst_allowance"].abs().min())
print(df_sklearn["current_qdelay"].abs().min())
print(df_sklearn["qdelay_old"].abs().min())
print(df_sklearn["accu_prob"].abs().min())
print(df_sklearn["measurement_start"].abs().min())
print(df_sklearn["tot_pkts"].abs().min())
print(df_sklearn["tot_bytes"].abs().min())
print(df_sklearn["length"].abs().min())
print(df_sklearn["len_bytes"].abs().min())
print(df_sklearn["drops"].abs().min())
print(df_sklearn["ecn"].abs().min())

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0


In [11]:
#@title Custom Gym Environment
observations = ['burst_allowance',
 'drop_prob',
 'current_qdelay',
 'qdelay_old',
 'accu_prob',
 'measurement_start',
 'tot_pkts',
 'tot_bytes',
 'length',
 'len_bytes',
 'drops',
 'ecn']

observations=list(df_sklearn.columns)

actions=['new_drop_prob']

states_record=[]

states_dict={}





def tcp_obs_space():
    lower_obs_bound = {
        'burst_allowance': - np.inf,
        'drop_prob': - np.inf,
        'current_qdelay': - np.inf,
        'qdelay_old': - np.inf,
        'accu_prob': - np.inf,
        'measurement_start': - np.inf,
        'tot_pkts': - np.inf,
        'tot_bytes': - np.inf,
        'length': - np.inf,
        'len_bytes': - np.inf,
        'drops': - np.inf,
        'ecn': - np.inf
        
    }

    higher_obs_bound = {
        'burst_allowance': np.inf,
        'drop_prob': np.inf,
        'current_qdelay': np.inf,
        'qdelay_old': np.inf,
        'accu_prob': np.inf,
        'measurement_start': np.inf,
        'tot_pkts': np.inf,
        'tot_bytes': np.inf,
        'length': np.inf,
        'len_bytes': np.inf,
        'drops': np.inf,
        'ecn': np.inf
        
    }

    low = np.array([lower_obs_bound[o] for o in observations])
    high = np.array([higher_obs_bound[o] for o in observations])
    shape = (len(observations),)
    return gym.spaces.Box(low,high,shape)


def tcp_action_space():
    lower_obs_bound = {
        'new_drop_prob': - np.inf,
    }
    higher_obs_bound = {
        'new_drop_prob': np.inf,
    }

    low = np.array([lower_obs_bound[o] for o in actions])
    high = np.array([higher_obs_bound[o] for o in actions])
    shape = (len(observations),)
    return gym.spaces.Box(low,high,shape)

class gym_env():
    """ """
    def __init__(self,filename):

        self.filename=filename

        
      
      

        

        # self.rate=df["rate"][0]
        # self.total_rate=df["total_rate"][0]
        # self.sent=df["sent"][0]
        # self.lost= df["lost"][0]
        # self.util= df["lost"][0]
        # self.old_util= df["old_util"][0]
        # self.thpt= df["thpt"][0]
        # self.loss_rate= df["loss_rate"][0]
        self.observations = ['burst_allowance','drop_prob','current_qdelay','qdelay_old','accu_prob',
                             'measurement_start','tot_pkts','tot_bytes','length','len_bytes','drops','ecn']

        
        self.observation_space = tcp_obs_space()

        self.count=0

        self.state_nos=0
        self.temp_states_list=[]
        
        self.action_space = spaces.Box(low=-np.inf,high=np.inf,dtype=np.float32,shape=(1,))
        

    

    def observation(self):
        return np.array([self.state[o] for o in self.observations])

    def reward(self):
        return self.state['current_qdelay']

    """ reset env, return the initial state  """
    def reset(self):
        time.sleep(1)
        # self.state={
        # 'rate': 0,
        # 'total_rate': 0,
        # 'sent': 0,
        # 'lost': 0,
        # 'util': 0,
        # 'old_util': 0,
        # 'thpt': 0,
        # 'loss_rate': 0,
        # }
        # self.state={
        # 'rate': df["rate"][random.randint(0, df.shape[0])],
        # 'total_rate': df["total_rate"][random.randint(0, df.shape[0])],
        # 'sent': df["sent"][random.randint(0, df.shape[0])],
        # 'lost': df["lost"][random.randint(0, df.shape[0])],
        # 'util': df["util"][random.randint(0, df.shape[0])],
        # 'old_util': df["old_util"][random.randint(0, df.shape[0])],
        # 'thpt': df["thpt"][random.randint(0, df.shape[0])],
        # 'loss_rate': df["loss_rate"][random.randint(0, df.shape[0])]
        # }

        rindex=random.randint(0, df_sklearn.shape[0]-1)

        
        self.state={
            'burst_allowance': df_sklearn.iloc[rindex]["burst_allowance"],
            'drop_prob': df_sklearn.iloc[rindex]["drop_prob"],
            'current_qdelay': df_sklearn.iloc[rindex]["current_qdelay"],
            'qdelay_old': df_sklearn.iloc[rindex]["qdelay_old"],
            'accu_prob': df_sklearn.iloc[rindex]["accu_prob"],
            'measurement_start': df_sklearn.iloc[rindex]["measurement_start"],
            'tot_pkts': df_sklearn.iloc[rindex]["tot_pkts"],
            'tot_bytes': df_sklearn.iloc[rindex]["tot_bytes"],
            'length': df_sklearn.iloc[rindex]["length"],
            'len_bytes': df_sklearn.iloc[rindex]["len_bytes"],
            'drops': df_sklearn.iloc[rindex]["drops"],
            'ecn': df_sklearn.iloc[rindex]["ecn"]
        }
        print("reset happened")
        print(df_sklearn.iloc[rindex]['current_qdelay'])
        self.state_nos=0
        self.temp_states_list=[]
        self.count=self.count+1

        states_record.append(self.state)
        self.temp_states_list.append(self.state)
        return self.observation()
        

    """ action = [sub1_buff_size, sub2_buff_size] """
    def step(self, action):
        # A = [self.fd, action[0], action[1]]
        # mpsched.set_seg(A)
        time.sleep(2)
        done = False
        
        
        #state_nxt = mpsched.get_sub_info(self.fd)

        input=action[0]

        if(math.isnan(input) or input<0):
          input=0
          print("step happened")
          print("Wrong action value")
          print(input)
          return self.observation(), self.reward(), done, {}


        df_sort = df_sklearn.iloc[(df_sklearn['drop_prob']-input).abs().argsort()[:1]]



        state_nxt={
            'burst_allowance': df_sort["burst_allowance"].values[0],
            'drop_prob': df_sort["drop_prob"].values[0],
            'current_qdelay': df_sort["current_qdelay"].values[0],
            'qdelay_old': df_sort["qdelay_old"].values[0],
            'accu_prob': df_sort["accu_prob"].values[0],
            'measurement_start': df_sort["measurement_start"].values[0],
            'tot_pkts': df_sort["tot_pkts"].values[0],
            'tot_bytes': df_sort["tot_bytes"].values[0],
            'length': df_sort["length"].values[0],
            'len_bytes': df_sort["len_bytes"].values[0],
            'drops': df_sort["drops"].values[0],
            'ecn': df_sort["ecn"].values[0]
        }

        

        
        
        if len(state_nxt) == 0:
            done = True


        self.state_nos=self.state_nos+1

        if(self.state_nos>50):
            done=True

       
        if self.state['current_qdelay']==(df_sklearn["current_qdelay"].max()+1):
            done = True

        # if(df_sort["total_rate"].values[0]==470000):
        #     rindex=random.randint(0, df_temp.shape[0]-1)

        #     df_temp.iloc[rindex]
        #     self.state={
        #     'rate': df_temp.iloc[rindex]["rate"],
        #     'total_rate': df_temp.iloc[rindex]["total_rate"],
        #     'sent': df_temp.iloc[rindex]["sent"],
        #     'lost': df_temp.iloc[rindex]["lost"],
        #     'util': df_temp.iloc[rindex]["util"],
        #     'old_util': df_temp.iloc[rindex]["old_util"],
        #     'thpt': df_temp.iloc[rindex]["thpt"],
        #     'loss_rate': df_temp.iloc[rindex]["loss_rate"]
        #     }

        if done==True:              
            states_dict[str(self.count)]=self.temp_states_list
            self.temp_states_list=[]

        print("step happened")
        print("action valu recieved")
        print(input)
        print(state_nxt)
        print(self.state_nos)
        print()
        print()
        print()


        self.state=state_nxt

        states_record.append(self.state)
        self.temp_states_list.append(self.state)


        
        return self.observation(), self.reward(), done, {}


In [12]:
env = gym_env(filename="rldata.txt")


state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
action_bound = env.action_space.high[0]

In [13]:
action_bound

inf

In [14]:
env.reset()

reset happened
0.052631578947368425


array([0.        , 0.01158717, 0.05263158, 0.05263158, 0.08079799,
       0.60755853, 0.07775164, 0.07786177, 0.03543307, 0.03543307,
       0.        , 1.        ])

In [15]:
# import wandb
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Lambda

import gym
import argparse
import numpy as np
from threading import Thread
from multiprocessing import cpu_count
tf.keras.backend.set_floatx('float64')
# wandb.init(name='A3C', project="deep-rl-tf2")



args_gamma=0.99
args_update_interval=5
args_actor_lr=0.0005
args_critic_lr=0.001




CUR_EPISODE = 0

class Actor:
    def __init__(self, state_dim, action_dim, action_bound, std_bound):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.action_bound = action_bound
        self.std_bound = std_bound
        self.model = self.create_model()
        self.opt = tf.keras.optimizers.Adam(args_actor_lr)
        self.entropy_beta = 0.01

    def create_model(self):
        state_input = Input((self.state_dim,))
        dense_1 = Dense(32, activation='relu')(state_input)
        dense_2 = Dense(32, activation='relu')(dense_1)
        out_mu = Dense(self.action_dim, activation='tanh')(dense_2)
        mu_output = Lambda(lambda x: x * self.action_bound)(out_mu)
        std_output = Dense(self.action_dim, activation='softplus')(dense_2)
        return tf.keras.models.Model(state_input, [mu_output, std_output])

    def get_action(self, state):
        state = np.reshape(state, [1, self.state_dim])
        mu, std = self.model.predict(state)
        # print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
        # print("state",end=",")
        # print(state)
        # print("mu",end=",")
        # print(mu)
        # print("std",end=",")
        # print(std)
        # print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
        mu, std = mu[0], std[0]
        
        return np.random.normal(mu, std, size=self.action_dim)

    def log_pdf(self, mu, std, action):
        std = tf.clip_by_value(std, self.std_bound[0], self.std_bound[1])
        var = std ** 2
        log_policy_pdf = -0.5 * (action - mu) ** 2 / \
            var - 0.5 * tf.math.log(var * 2 * np.pi)
        return tf.reduce_sum(log_policy_pdf, 1, keepdims=True)

    def compute_loss(self, mu, std, actions, advantages):
        log_policy_pdf = self.log_pdf(mu, std, actions)
        loss_policy = log_policy_pdf * advantages
        return tf.reduce_sum(-loss_policy)

    def train(self, states, actions, advantages):
        with tf.GradientTape() as tape:
            mu, std = self.model(states, training=True)
            loss = self.compute_loss(mu, std, actions, advantages)
        grads = tape.gradient(loss, self.model.trainable_variables)
        self.opt.apply_gradients(zip(grads, self.model.trainable_variables))
        return loss


class Critic:
    def __init__(self, state_dim):
        self.state_dim = state_dim
        self.model = self.create_model()
        self.opt = tf.keras.optimizers.Adam(args_critic_lr)

    def create_model(self):
        return tf.keras.Sequential([
            Input((self.state_dim,)),
            Dense(32, activation='relu'),
            Dense(32, activation='relu'),
            Dense(16, activation='relu'),
            Dense(1, activation='linear')
        ])

    def compute_loss(self, v_pred, td_targets):
        mse = tf.keras.losses.MeanSquaredError()
        return mse(td_targets, v_pred)

    def train(self, states, td_targets):
        with tf.GradientTape() as tape:
            v_pred = self.model(states, training=True)
            assert v_pred.shape == td_targets.shape
            loss = self.compute_loss(v_pred, tf.stop_gradient(td_targets))
        grads = tape.gradient(loss, self.model.trainable_variables)
        self.opt.apply_gradients(zip(grads, self.model.trainable_variables))
        return loss


class Agent:
    def __init__(self, env_name):
        env = gym.make(env_name)
        if use_custom:
            env = gym_env(filename="rldata.txt")
        self.env_name = env_name       
        
        self.state_dim = env.observation_space.shape[0]
        self.action_dim = env.action_space.shape[0]
        self.action_bound = 1.000
        
        
        
        
        
        self.std_bound = [1e-2, 1.0]

        self.global_actor = Actor(
            self.state_dim, self.action_dim, self.action_bound, self.std_bound)
        self.global_critic = Critic(self.state_dim)
        self.num_workers = cpu_count()

    def train(self, max_episodes=1):
        workers = []

        for i in range(self.num_workers):
            env = gym.make(self.env_name)
            if use_custom:
                env = gym_env(filename="rldata.txt")
            workers.append(WorkerAgent(
                env, self.global_actor, self.global_critic, max_episodes))

        for worker in workers:
            worker.start()

        for worker in workers:
            worker.join()


class WorkerAgent(Thread):
    def __init__(self, env, global_actor, global_critic, max_episodes):
        Thread.__init__(self)
        self.env = env
        self.state_dim = self.env.observation_space.shape[0]
        self.action_dim = self.env.action_space.shape[0]
        self.action_bound = 1.000
        self.std_bound = [1e-2, 1.0]

        self.max_episodes = max_episodes
        self.global_actor = global_actor
        self.global_critic = global_critic
        self.actor = Actor(self.state_dim, self.action_dim,
                           self.action_bound, self.std_bound)
        self.critic = Critic(self.state_dim)

        self.actor.model.set_weights(self.global_actor.model.get_weights())
        self.critic.model.set_weights(self.global_critic.model.get_weights())

    def n_step_td_target(self, rewards, next_v_value, done):
        td_targets = np.zeros_like(rewards)
        cumulative = 0
        if not done:
            cumulative = next_v_value

        for k in reversed(range(0, len(rewards))):
            cumulative = args_gamma * cumulative + rewards[k]
            td_targets[k] = cumulative
        return td_targets

    def advatnage(self, td_targets, baselines):
        return td_targets - baselines

    def list_to_batch(self, list):
        batch = list[0]
        for elem in list[1:]:
            batch = np.append(batch, elem, axis=0)
        return batch

    def train(self):
        global CUR_EPISODE

        while self.max_episodes >= CUR_EPISODE:
            state_batch = []
            action_batch = []
            reward_batch = []
            episode_reward, done = 0, False

            state = self.env.reset()
            

            while not done:
                # self.env.render()
                if use_custom:
                    action = self.actor.get_action(state)
                else:
                    action = self.actor.get_action(state)[0]
                
                print("Original action :",end="")
                print(action)
                action = np.clip(action, 0, self.action_bound)
                
                #print(self.env.step(action))
                
                if use_custom:
                    next_state, reward, done, info = self.env.step(action)
                else:
                    next_state, reward, done, info,_ = self.env.step(action)                    

                state = np.reshape(state, [1, self.state_dim])
                action = np.reshape(action, [1, 1])
                next_state = np.reshape(next_state, [1, self.state_dim])
                reward = np.reshape(reward, [1, 1])

                state_batch.append(state)
                action_batch.append(action)
                reward_batch.append(reward)
                print("State :",end="")
                print(state)
                print("action :",end="")
                print(action)
                print("reward :",end="")
                print(reward)

                if len(state_batch) >= args_update_interval or done:
                    states = self.list_to_batch(state_batch)
                    actions = self.list_to_batch(action_batch)
                    rewards = self.list_to_batch(reward_batch)

                    next_v_value = self.critic.model.predict(next_state)
                    td_targets = self.n_step_td_target(
                        (rewards+8)/8, next_v_value, done)
                    advantages = td_targets - self.critic.model.predict(states)

                    actor_loss = self.global_actor.train(
                        states, actions, advantages)
                    critic_loss = self.global_critic.train(
                        states, td_targets)

                    self.actor.model.set_weights(
                        self.global_actor.model.get_weights())
                    self.critic.model.set_weights(
                        self.global_critic.model.get_weights())

                    state_batch = []
                    action_batch = []
                    reward_batch = []
                    td_target_batch = []
                    advatnage_batch = []

                episode_reward += reward[0][0]
                state = next_state[0]

            print('EP{} EpisodeReward={}'.format(CUR_EPISODE, episode_reward))
            # wandb.log({'Reward': episode_reward})
            CUR_EPISODE += 1

    def run(self):
        self.train()


def main():
    env_name = 'Pendulum-v1'
    agent = Agent(env_name)
    agent.train()


if __name__ == "__main__":
    main()

reset happened
0.0
reset happened
0.07894736842105263
reset happened
0.5
reset happened
0.15789473684210525
reset happened
0.052631578947368425
reset happened
0.4210526315789474
reset happened
0.052631578947368425
reset happened
0.0
reset happened
0.052631578947368425
reset happened
0.13157894736842105
reset happened
0.2368421052631579
reset happened
0.10526315789473685
1/1 [==============================] - ETA: 0sOriginal action :[-0.01231391]
Original action :[0.45365171]
1/1 [==============================] - 1s 557ms/step
Original action :[0.26676587]
1/1 [==============================] - 1s 600ms/step
Original action :[1.12847977]
Original action :[-1.02964051]
Original action :[0.2082003]
Original action :[-0.69868575]
Original action :[-1.08070062]
Original action :[-0.10437572]
1/1 [==============================] - 1s 584ms/step
Original action :[2.12452428]
Original action :[-0.11670772]
step happened
action valu recieved
0.7221816798622125
{'burst_allowance': 0.0, 'drop_pr